In [14]:
results=[{'delivery': 'sentence',
  'inspector': 'specs_of',
  'part': 'predicate',
  'provider': 'default',
  'value': {'category': 'correct', 'pos': 'v', 'word': '直した。/直す'}},
 {'delivery': 'slot',
  'inspector': 'ins_date',
  'part': '修飾',
  'provider': 'duckling',
  'value': [{'body': '8',
             'dim': 'number',
             'end': 1,
             'latent': False,
             'start': 0,
             'value': {'type': 'value', 'value': 8}}]},
 {'delivery': 'sentence',
  'inspector': 'kind_of',
  'part': 'ヲ',
  'provider': 'default',
  'value': {'category': 'artifact', 'pos': 'n', 'word': '流しを/流し'}},
 {'delivery': 'slot',
  'inspector': 'series',
  'part': 'defined',
  'provider': 'events',
  'value': {'_count': 'ins_date:$[0].value.value',
            'action': 'specs_of:category',
            'object': 'kind_of:category'}}]

from jsonpath_ng import jsonpath, parse

val_list = [r['value'] for r in results if r['inspector'] == 'series']
tags={}
fields={}
for val in val_list:
    print(f"{val}")    
    for k,expr in val.items():        
        if k.startswith('_'):
            key=k[1:]
            target_c=fields
        else:
            key=k
            target_c=tags
        ins_name,val_expr=expr.split(':')
        head=next(item for item in results if item['inspector']==ins_name)
        vals=head['value']
        if val_expr.startswith('$'):
            jsonpath_expr = parse(val_expr)
            eval_val=next(match.value for match in jsonpath_expr.find(vals))
        else:
            eval_val=vals[val_expr]
        target_c[key]=eval_val
print('tags', tags)
print('fields', fields)

{'_count': 'ins_date:$[0].value.value', 'action': 'specs_of:category', 'object': 'kind_of:category'}
tags {'action': 'correct', 'object': 'artifact'}
fields {'count': 8}


In [19]:
series_ins=[r for r in results if r['inspector'] == 'series']
series=series_ins[0]
series

{'delivery': 'slot',
 'inspector': 'series',
 'part': 'defined',
 'provider': 'events',
 'value': {'_count': 'ins_date:$[0].value.value',
  'action': 'specs_of:category',
  'object': 'kind_of:category'}}

In [16]:
from influxdb import InfluxDBClient

client = InfluxDBClient('localhost', 8086, 'root', '', 'events')
print(client.get_list_database())
# client.create_database("events")

[{'name': '_internal'}, {'name': 'testDB'}, {'name': 'NOAA_water_database'}, {'name': 'pyexample'}, {'name': 'events'}]


In [21]:
result = client.query("show measurements;")
print("Result: {0}".format(result))

Result: ResultSet({'('measurements', None)': [{'name': 'events'}]})


In [20]:
json_body = [
    {
        "measurement": f"{series['provider']}",
        "tags": tags,
        #"time": "2017-03-12T22:00:00Z",
        "fields": fields
    }
]
client.write_points(json_body)

True

In [25]:
qs=f"select * from {series['provider']};"
result = client.query(qs)    
print("Result: {0}".format(result))

Result: ResultSet({'('events', None)': [{'time': '2020-03-04T19:33:43.248733Z', 'action': 'correct', 'count': 8, 'object': 'artifact'}]})


In [11]:
from jsonpath_ng import jsonpath, parse
vals=[{'body': '8',
             'dim': 'number',
             'end': 1,
             'latent': False,
             'start': 0,
             'value': {'type': 'value', 'value': 8}}]
jsonpath_expr = parse('$[0].value.value')
next(match.value for match in jsonpath_expr.find(vals))

8